In [114]:
from __future__ import print_function

import pandas as pd
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import plotly
from plotly.offline import iplot
import plotly.express as px
import plotly.graph_objects as go
import folium
import requests
from bs4 import BeautifulSoup

In [115]:
url = 'https://covid19.ncdc.gov.ng/'

headers = {'User-Agent': 'Mozilla'}

response = requests.get(url, headers = headers)
response.status_code

soup = BeautifulSoup(response.content, 'html.parser')
stat_tab = soup.find_all("table", id='custom1')
#stat_tab = stat_tab[0]

data = pd.read_html(str(stat_tab))[0]

In [116]:
confirmed = data["No. of Cases (Lab Confirmed)"]
death = data["No. of Deaths"]
active = data["No. of Cases (on admission)"]
state = data["States Affected"]
discharged = data["No. Discharged"]

In [117]:
st = ["ABIA","ADAMAWA","AKWA IBOM","ANAMBRA","BAUCHI","BAYELSA","BENUE","BORNO","CROSS RIVER","DELTA","EBONYI","EDO",
      "EKITI","ENUGU","FCT","GOMBE","IMO","JIGAWA","KADUNA","KANO","KATSINA","KEBBI","KOGI","KWARA","LAGOS","NASARAWA",
      "NIGER","OGUN","ONDO","OSUN","OYO","PLATEAU","RIVERS","SOKOTO","TARABA","YOBE","ZAMFARA"]

lat = [5.5225,9.2766,4.9576,6.1964,10.4102,4.6858,7.3574,11.8371,5.2701,5.5294,6.2722,6.4312,7.6712,6.45,8.9353,10.3192,5.5462,12.3199,10.5088,11.9992,12.9866,11.6586,7.7893,8.9665,6.5214,8.5608,9.8337,7.1393,6.9277,7.5323,8.1544,9.1822,4.6656,13.0404,8.2089,12.3265,12.1674]

long = [7.493,12.3838,7.84,6.9834,9.9639,6.0158,8.7588,13.1485,8.5415,5.8015,8.0953,5.8944,5.318,7.507,7.23,11.2017,7.0865,9.5637,7.4382,8.5323,7.624,4.0848,6.7085,4.6988,3.2942,8.1687,5.5481,3.3538,5.066,4.5152,3.6991,9.5027,7.0309,5.2426,10.9726,11.5391,6.2946]

## INTERACTIVE COVID-19 DASHBOARD MONITORING NIGERIA CASES

The COVID-19 pandemic in Nigeria is part of the worldwide pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The first confirmed case in Nigeria was announced on 27 of February, 2020, when an Italian citizen in Lagos tested positive for the virus. The Italian citizen who works in Nigeria had returned on 25 of February from Milan, Italy through the Murtala Muhammed International Airport, fell ill on 26 February and was transferred to Lagos State biosecurity facilities for isolation and testing.
On 9 March 2020, a second case of the virus was reported in Ewekoro, Ogun State, a Nigerian citizen who had contact with the Italian citizen

In [118]:
webMap = folium.Map(location=[9.08,7.53], tiles="CartoDB positron", zoom_start=6, max_zoom=6, min_zoom=6)
confirm = np.array(confirmed)
num_st = len(st)
num_state = len(state)

def size(i):
    global sx
    for j in range (len(state)):
        state2 = str(state[j]).upper()
        if st[i] == state2:
            value = confirm[j]
            ac = active[j]
            dth = death[j]
            dc = discharged[j]
            if value > 1000:
                sx = 25
            elif 1000 > value > 500:
                sx = 15
            elif 500 > value > 200:
                sx = 10
            elif 200 > value > 1:
                sx = 5
            else:
                sx = 0
        else:
            pass
    return sx

def val(i,d):
    global d_out
    for j in range (len(state)):
        state2 = str(state[j]).upper()
        if st[i] == state2:
            if d == 'cc':
                d_out = confirm[j]
            if d == ac:
                d_out = active[j]
            if d == dth:
                d_out = death[j]
            if d == dc:
                d_out = discharged[j]
        else:
            pass
    return d_out



for i in range (num_st):
    folium.CircleMarker(
        location=[lat[i],long[i]], 
        fill = True,
        radius = size(i),
        fill_color = "red",
        color = "red",
        tooltip = "%s<br><br>Active: %s<br>Recovered: %s<br>Deaths: %s,"%(st[i],val(i,ac),val(i,dc),val(i,dth))
        ).add_to(webMap)

    
webMap

Filtering through the 36 states to find the top 10 states in each category

In [119]:
def query(select):
    if select == "confirmed":
        fig = px.scatter(data.head(10), x="States Affected", y="No. of Cases (Lab Confirmed)", size="No. of Cases (Lab Confirmed)", 
                         color="States Affected", hover_name="States Affected", size_max=60, title="10 States with the Highest Confirmed Cases")
        fig.update_layout()
    elif select == "active":
        fig = px.scatter(data.head(10).sort_values("No. of Cases (on admission)", ascending=False), 
                         x="States Affected", y="No. of Cases (on admission)", size="No. of Cases (on admission)", color="States Affected", hover_name="States Affected", size_max=50, title="10 States with the Highest Active Cases")
        fig.update_layout()
    elif select == "discharged":
        fig = px.scatter(data.head(10).sort_values("No. Discharged", ascending=False), 
                         x="States Affected", y="No. Discharged", size="No. Discharged", color="States Affected", hover_name="States Affected", size_max=40, title="10 States with the Highest Recovery")
        fig.update_layout()
    elif select == "death":
        fig = px.scatter(data.head(10).sort_values("No. of Deaths", ascending=False), 
                         x="States Affected", y="No. of Deaths", size="No. of Deaths", color="States Affected", hover_name="States Affected", size_max=30, title="10 States with the Highest Number of Deaths")
        fig.update_layout()
    fig.show()
    
interact(query, select=["confirmed","active","discharged","death"]);

interactive(children=(Dropdown(description='select', options=('confirmed', 'active', 'discharged', 'death'), v…

In [120]:
def Search_by_case_type (case):
    if case == "Confirmed Cases":
        trace = go.Bar(x=state, y=confirmed, name='Confirmed Cases')
        layout = go.Layout(title = "Total Confirmed Case in Each State")
    if case == "Active Case":
        trace = go.Bar(x=state, y=active, name='Total Active Case in Each State')
        layout = go.Layout(title = "Active Cases")
    if case == "Discharged":
        trace = go.Bar(x=state, y=discharged, name='Discharged')
        layout = go.Layout(title = "Total Discharged in Each State")
    if case == "Death":
        trace = go.Bar(x=state, y=death, name='Deaths')
        layout = go.Layout(title = "Death Toll")

    figure = go.Figure(data = trace, layout = layout)
    return figure.show()

interact(Search_by_case_type, case=["Confirmed Cases", "Active Case", "Discharged", "Death"]);

interactive(children=(Dropdown(description='case', options=('Confirmed Cases', 'Active Case', 'Discharged', 'D…

In [121]:
total_active = np.sum(np.array(active))
recovered  = np.sum(np.array(discharged))
total_death = np.sum(np.array(death))
total_confirmed = np.sum(np.array(confirmed))


def select_state(Select_State):
    st = Select_State.lower()
    if st == "nigeria":
        ac = total_active
        dc = recovered
        dth = total_death
        print("Total Confirmed: ", total_confirmed)
        report = ['Active Case', 'Death', 'Recovered']
        value = [ac, dth, dc]
        trace = go.Pie(labels = report, values = value)
        data = [trace]
    for i in range(len(state)):
        state1 = str(state[i]).lower()
        if state1 == st:
            print("Total confirmed: ", confirmed[i])
            ac = active[i]
            dc = discharged[i]
            dth = death[i]
            report = ['Active Case', 'Death', 'Recovered']
            value = [ac, dth, dc]
            trace = go.Pie(labels = report, values = value, title = "")
            data = [trace]
        else:
            pass
    fig = go.Figure(data = data)
    iplot(fig)
        
interact(select_state, Select_State=state);

interactive(children=(Dropdown(description='Select_State', options=('Lagos', 'FCT', 'Kano', 'Oyo', 'Rivers', '…

In [122]:
def highlight_col(x):
    r = "background-color: red"
    gy = "background-color: grey"
    b = "background-color: blue"
    g = "background-color: green"
    temp_data = pd.DataFrame("", index=x.index, columns = x.columns)
    temp_data.iloc[:,1] = gy
    temp_data.iloc[:,2] = b
    temp_data.iloc[:,3] = g
    temp_data.iloc[:,4] = r
    return temp_data

data.style.apply(highlight_col, axis=None)

,States Affected,No. of Cases (Lab Confirmed),No. of Cases (on admission),No. Discharged,No. of Deaths
0,Lagos,8576,7006,1444,126
1,FCT,1567,1051,486,30
2,Kano,1190,428,712,50
3,Oyo,912,621,282,9
4,Rivers,874,452,391,31
5,Edo,784,556,197,31
6,Ogun,646,270,361,15
7,Kaduna,580,268,302,10
8,Delta,501,360,124,17
9,Borno,468,75,362,31
